In [20]:
import pybullet as p
import time
import numpy as np
import pybullet_data

In [21]:
cos = np.cos
sin = np.sin
def Trotx(ang):
    T = np.array([[1, 0, 0, 0],
                   [0, cos(ang), -sin(ang), 0],
                   [0, sin(ang),  cos(ang), 0],
                   [0, 0, 0, 1]])
    return T
def Troty(ang):
    T = np.array([[cos(ang), 0, sin(ang),0],
                   [0, 1, 0,0],
                   [-sin(ang), 0, cos(ang),0],
                  [0, 0, 0, 1]])
    return T
def Trotz(ang):
    T = np.array([[cos(ang), -sin(ang), 0,0],
                   [sin(ang), cos(ang), 0,0],
                   [0,0,1,0],
                   [0,0,0,1]])
    return T

In [22]:
# Cinemática del mini cheetah
def Tdh(d, th, a, alpha):
    cth = np.cos(th);    sth = np.sin(th)
    ca = np.cos(alpha);  sa = np.sin(alpha)
    Tdh = np.array([[cth, -ca*sth,  sa*sth, a*cth],
                    [sth,  ca*cth, -sa*cth, a*sth],
                    [0,        sa,     ca,      d],
                    [0,         0,      0,      1]])
    return Tdh

# Cinemática directa del robot
def get_pose(q,side=1):
    """ Retorna los sistemas de referencia de cada eslabón con respecto a la base
    """
    #side 1 is left and -1 is right

    L1 = 0.062; L2 = 0.209; L3 = 0.18

    # Transformaciones homogéneas de DH
    T01 = Tdh( 0, q[0], side*L1, 0)
    T12 = Tdh( 0, -np.pi/2, 0, -np.pi/2)
    T23 = Tdh( 0, -q[1], L2, 0)
    T34 = Tdh( 0, -q[2], L3, 0)
    # Efector final con respecto a la base
    T04 = T01.dot(T12).dot(T23).dot(T34)
    
    return T04
    
    
def jacobiano(q, delta=1e-4, side=1):
    # Alocación de memoria
    J = np.zeros((3,4))
    # Cinemática directa(sin incrementos)
    f = get_pose(q,side)[0:3,3]
    for i in range(4):
        qincr = np.copy(q)
        qincr[i] = qincr[i] + delta
        J[:,i] = (get_pose(qincr)[0:3,3]-f)/delta
    return J

### Inicialización

Se utilizará el robot laikago, cuyo URDF se encuentra por defecto en `laikago/laikago_toes.urdf`

In [23]:
# Conexión al simulador físico
physicsClient = p.connect(p.GUI)
# Ruta a los archivos por defecto de PyBullet 
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Resetear el simulador 
p.resetSimulation()
# Gravedad del simulador
p.setGravity(0,0,-9.81)
# Punto de vista inicial
p.resetDebugVisualizerCamera(cameraDistance=1.2, cameraYaw=135, cameraPitch=-20, 
                                 cameraTargetPosition=[0,0,0])
# Añadir un plano
plano = p.loadURDF("plane.urdf")
# Añadir el robot mini cheetah
# Posición y orientación inicial
start_pos = [0, 0, 0.45]
start_orient = p.getQuaternionFromEuler([0,0,0])
# Robot cuadrúpedo
robot = p.loadURDF("mini_cheetah/mini_cheetah.urdf", start_pos, start_orient)


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Iris(R) Plus Graphics (ICL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) Iris(R) Plus Graphics (ICL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


# cinematica

In [24]:
# Índices de las articulaciones actuadas
id_joints = [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14]

In [25]:
# Se añade una pequeña pelotita para verificar visualmente la cinemática

ball = p.loadURDF("sphere_small.urdf", [2,2,2])

ball1 = p.loadURDF("sphere_small.urdf", [2,2,2])

ball2 = p.loadURDF("sphere_small.urdf", [2,2,2])

ball3 = p.loadURDF("sphere_small.urdf", [2,2,2])
ini=[0,0,0]


In [32]:
p.resetBasePositionAndOrientation(ball, ini, [0,0,0,1])
p.resetBasePositionAndOrientation(ball1, ini, [0,0,0,1])
p.resetBasePositionAndOrientation(ball2, ini, [0,0,0,1])
p.resetBasePositionAndOrientation(ball3, ini, [0,0,0,1])

In [39]:
# Cinemática directa del robot
def get_pose(q,side=1):
    """ Retorna los sistemas de referencia de cada eslabón con respecto a la base
    """
    #side 1 is left and -1 is right

    L1 = 0.062; L2 = 0.209; L3 = 0.18

    # Transformaciones homogéneas de DH
    T01 = Tdh( 0, q[0], side*L1, 0)
    T12 = Tdh( 0, -np.pi/2, 0, -np.pi/2)
    T23 = Tdh( 0, -q[1], L2, 0)
    T34 = Tdh( 0, -q[2], L3, 0)
    # Efector final con respecto a la base
    T04 = T01.dot(T12).dot(T23).dot(T34)
    
    return T04
    
# Obtener la posición y orientación actuales de la base del robot
robot_pos, robot_or = p.getBasePositionAndOrientation(robot)

print("Posición del robot (x,y,z):", np.round(robot_pos,3))
print("Orientación del robot (ex,ey,ez,w):", np.round(robot_or,3))

# Convertir el cuaternión a matriz de rotación 3x3
matrix_rot = np.array(p.getMatrixFromQuaternion(robot_or)).reshape(3, 3)
#print("Matriz de rotación:\n", matrix_rot)

# Concatenar la matriz de rotación y el vector de posición del robot
T = np.hstack((matrix_rot, np.array(robot_pos).reshape(3, 1)))
# Agregar la última fila [0, 0, 0, 1]
T = np.vstack((T, np.array([0, 0, 0, 1])))


Teje=Trotx(np.pi/2) #CAMBIO DE ROTACION DEL EJE DE LA BASE FLOTANTE
T = T.dot(Teje)
print("Matriz de transformación homogénea T:\n", np.round(T,3))


#The positions and orientations of each leg can be calculated according 
# to the position and orientation of the robot's body.
#The position of the body is given by the position of the robot's base

c = np.cos(np.pi/2);    s = np.sin(np.pi/2)
L=0.19; w=0.049  #Distancia del centro de la base a cada pata
#TRANSFORMACIONES HOMOGÉNEAS DE CADA PATA RESPECTO LA BASE

#right front leg
Trf = T.dot(np.array([  [c,  0,  s,   L],
                        [0,  1,  0,   0],
                        [-s, 0,  c,   w],
                        [0,  0,  0,   1]]))

#right back leg
Trb = T.dot(np.array([  [c,  0,  s,  -L],
                        [0,  1,  0,   0],
                        [-s, 0,  c,   w],
                        [0,  0,  0,   1]]))

#left front leg
Tlf = T.dot(np.array([  [c,  0,  s,   L],
                        [0,  1,  0,   0],
                        [-s, 0,  c,   -w],
                        [0,  0,  0,   1]]))

#left back leg
Tlb = T.dot(np.array([  [c,  0,  s,   -L],
                        [0,  1,  0,   0],
                        [-s, 0,  c,   -w],
                        [0,  0,  0,   1]]))
                        
print("Matriz de transformación homogénea Trf:\n", np.round(Trf,3))
print("Matriz de transformación homogénea Trb:\n", np.round(Trb,3))
print("Matriz de transformación homogénea Tlf:\n", np.round(Tlf,3))
print("Matriz de transformación homogénea Tlb:\n", np.round(Tlb,3))

# Configuración inicial del robot (impuesta arbitrariamente)
q0 = [0, 0, -0, 0, 0, 0, 0, 0, -0, 0, 0, -0]
for i, joint in enumerate(id_joints):
    p.resetJointState(robot, joint, q0[i])
    
q = [state[0] for state in p.getJointStates(robot, id_joints)]

# DH de cada pata
T04_rf=get_pose(q[0:3],side=-1)
T04_lf=get_pose(q[3:6],side=1)
T04_rb=get_pose(q[6:9],side=-1)
T04_lb=get_pose(q[9:12],side=1)

#transformación respecto a un punto de referencia (eje original)
TF_lf=Tlf.dot(T04_lf)
print("Matriz de transformación homogénea TF leftfront limb:\n", np.round(TF_lf,3))

TF_lb=Tlb.dot(T04_lb)
print("Matriz de transformación homogénea TF leftback limb:\n", np.round(TF_lb,3))

TF_rf=Trf.dot(T04_rf)
print("Matriz de transformación homogénea TF rightfront limb:\n", np.round(TF_rf,3))

TF_rb=Trb.dot(T04_rb)
print("Matriz de transformación homogénea TF rightback limb:\n", np.round(TF_rb,3))


p.resetBasePositionAndOrientation(ball, TF_lf[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball1, TF_lb[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball2, TF_rf[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball3, TF_rb[0:3,3], [0,0,0,1])


Posición del robot (x,y,z): [-0.005 -0.021  0.385]
Orientación del robot (ex,ey,ez,w): [-0.007 -0.007 -0.002  1.   ]
Matriz de transformación homogénea T:
 [[ 1.    -0.014 -0.005 -0.005]
 [-0.005  0.013 -1.    -0.021]
 [ 0.014  1.     0.013  0.385]
 [ 0.     0.     0.     1.   ]]
Matriz de transformación homogénea Trf:
 [[ 0.005 -0.014  1.     0.185]
 [ 1.     0.013 -0.005 -0.071]
 [-0.013  1.     0.014  0.389]
 [ 0.     0.     0.     1.   ]]
Matriz de transformación homogénea Trb:
 [[ 0.005 -0.014  1.    -0.195]
 [ 1.     0.013 -0.005 -0.069]
 [-0.013  1.     0.014  0.383]
 [ 0.     0.     0.     1.   ]]
Matriz de transformación homogénea Tlf:
 [[ 0.005 -0.014  1.     0.186]
 [ 1.     0.013 -0.005  0.027]
 [-0.013  1.     0.014  0.387]
 [ 0.     0.     0.     1.   ]]
Matriz de transformación homogénea Tlb:
 [[ 0.005 -0.014  1.    -0.194]
 [ 1.     0.013 -0.005  0.029]
 [-0.013  1.     0.014  0.382]
 [ 0.     0.     0.     1.   ]]
Matriz de transformación homogénea TF leftfront limb:
 

# gravedad

In [31]:
# Hacer que el robot "baje" hasta el suelo (simulación dinámica)
for i in range(200):
    p.stepSimulation()
    time.sleep(0.005)

# Simulación cinemática

La simulación cinemática consiste en simplemente visualizar el comportamiento del robot a nivel cinemático, sin considerar las fuerzas o torques que ejercen los motores. Es algo similar a lo realizado por RViz en ROS.

In [21]:
# Desactivar la simulación dinámica
p.setRealTimeSimulation(0)

# Configuración inicial del robot (impuesta arbitrariamente)
q0 = 4*[0, 0, 0.5]
for i, joint in enumerate(id_joints):
    p.resetJointState(robot, joint, q0[i])

# Velocidad deseada (final) para cada articulación
dqdes = njoints*[0]
# Configuración articular deseada para cada articulación
qdes = 4*[0, 0, 1]
# Valores articulares actuales
q = np.array([state[0] for state in p.getJointStates(robot, id_joints)])

# Incremento en valores articulares
N = 200
delta = (qdes-q)/N

# Movimiento del robot (simulación cinemática)
dt = 0.005
for i in range(N):
    # Valores articulares deseados
    qtarget = q + delta*i
    # Envío directo de valores articulares deseados al robot (sin simulación dinámica)
    for i, joint in enumerate(id_joints):
        p.resetJointState(robot, joint, qtarget[i])
    time.sleep(dt)

In [22]:
# Asignación de la configuración inicial
# Configuración inicial del robot (impuesta arbitrariamente)
q0 = 4*[0, 0, 0.5]
for i, joint in enumerate(id_joints):
    p.resetJointState(robot, joint, q0[i])

### Simulación dinámica

In [23]:
# Activar simulación dinámica
p.setRealTimeSimulation(1)

# Velocidad deseada (final) para cada articulación
dqdes = njoints*[0]
# Configuración articular deseada para cada articulación
qdes = 4*[0, 0, 1]
# Valores articulares actuales
q = np.array([state[0] for state in p.getJointStates(robot, id_joints)])

# Incremento en valores articulares
N = 200
delta = (qdes-q)/N

# Movimiento del robot (simulación dinámica)
dt = 0.001
for i in range(N):
    qtarget = q + delta*i
    p.setJointMotorControlArray(robot, id_joints, p.POSITION_CONTROL,
                                qtarget, dqdes, positionGains = njoints*[0.03], 
                                velocityGains = njoints*[1])
    time.sleep(dt)

# Esperar 2 segundos más
time.sleep(2)
# Detener la simulación dinámica
p.setRealTimeSimulation(0)

In [24]:
# Activar simulación dinámica
p.setRealTimeSimulation(1)

# Velocidad deseada (final) para cada articulación
dqdes = njoints*[0]
# Configuración articular deseada para cada articulación
qdes = 4*[0, 0, 0.5]
# Valores articulares actuales
q = np.array([state[0] for state in p.getJointStates(robot, id_joints)])

# Incremento en valores articulares
N = 200
delta = (qdes-q)/N

# Movimiento del robot (simulación dinámica)
dt = 0.001
for i in range(N):
    qtarget = q + delta*i
    p.setJointMotorControlArray(robot, id_joints, p.POSITION_CONTROL,
                                qtarget, dqdes, positionGains = njoints*[0.03], 
                                velocityGains = njoints*[1])
    time.sleep(dt)

# Esperar 2 segundos más
time.sleep(2)
# Detener la simulación dinámica
p.setRealTimeSimulation(0)

# Desconexión

Es importante desconectarse de la simulación al finalizar la simulación.

In [40]:
p.disconnect()

numActiveThreads = 0
stopping threads
destroy semaphore
Thread with taskId 0 exiting
Thread TERMINATED
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
